In [2]:
import igraph as ig


track_network_path = "../network-parser/output/track_graph.graphml"
artist_network_path = "../network-parser/output/artist_graph.graphml"

# read graphml files
track_network = ig.Graph.Read_GraphML(track_network_path)
artist_network = ig.Graph.Read_GraphML(artist_network_path)

# force convert to undirected
track_network.to_undirected()
artist_network.to_undirected()

# force convert to simple
track_network.simplify()
artist_network.simplify()

# confirm
print(track_network.is_simple())
print(artist_network.is_simple())



True
True


# Plain Random Walk

In [4]:
import igraph as ig
import random
import json

# Step 1: Load the graph from GraphML
graph = ig.Graph.Read_GraphML(track_network_path)
graph.to_undirected()  # Ensure the graph is undirected

random.seed(42)

# Step 2: Random walk function avoiding tracks in user's playlist
def weighted_random_walk_exclude_user_pid(graph, start_node, user_pid, num_steps):
    walk = [start_node]
    current_node = start_node

    for _ in range(num_steps):
        neighbors = graph.neighbors(current_node, mode="all")

        # Exclude neighbors associated with the user's playlist ID
        valid_neighbors = [
            neighbor for neighbor in neighbors
            #if user_pid not in map(int, graph.vs[neighbor]["playlists"].split(","))
        ]
        if not valid_neighbors:
            break  # Stop if no valid neighbors

        edge_ids = [graph.get_eid(current_node, neighbor) for neighbor in valid_neighbors]
        weights = [graph.es[edge_id]["weight"] for edge_id in edge_ids]

        next_node = random.choices(valid_neighbors, weights=weights, k=1)[0]
        walk.append(next_node)
        current_node = next_node

    return walk

# Step 3: Recommendation function
def recommend_tracks_by_pid(graph, user_pid, user_track, num_steps=5, top_n=1):
    try:
        start_node = graph.vs.find(id=user_track).index
        print('i found it')
    except ValueError:
        # raise ValueError(f"Track '{user_track}' not found in the graph.")
        # print('nope')
        return None

    walk = weighted_random_walk_exclude_user_pid(graph, start_node, user_pid, num_steps)

    visit_counts = {}
    for node in walk:
        track_id = graph.vs[node]["id"]
        #if user_pid not in map(int, graph.vs[node]["playlists"].split(",")):
        visit_counts[track_id] = visit_counts.get(track_id, 0) + 1

    recommended_tracks = sorted(visit_counts, key=visit_counts.get, reverse=True)
    return recommended_tracks[:top_n]

# Step 4: Accuracy testing function
def test_recommendation_accuracy(graph, test_data_path, num_steps=5, top_n=1):
    with open(test_data_path, "r") as f:
        test_data = json.load(f)

    playlists = test_data["playlists"]
    total_tests = 0
    successful_tests = 0

    for playlist in playlists:
        user_pid = playlist["pid"]
        input_track_index = 0
        input_track = playlist["tracks"][input_track_index]["track_uri"]  # First track in the playlist
        ground_truth_tracks = {track["track_uri"] for track in playlist["tracks"]}
        
        try:
            while input_track_index < len(playlist["tracks"]):
                recommended_tracks = recommend_tracks_by_pid(
                    graph, user_pid, input_track, num_steps, top_n
                )
                if recommended_tracks is None:
                    input_track_index + 1
                    continue
                else:
                    print('find')
                    total_tests += 1
                # Check if any recommended track is in the ground truth tracks
                    if any(track in ground_truth_tracks for track in recommended_tracks):
                        successful_tests += 1
                        print('yep')
                    break
        except ValueError as e:
            print(e)  # Handle cases where the track is not found

    accuracy = successful_tests / total_tests
    print(f"Accuracy: {accuracy * 100:.2f}% ({successful_tests}/{total_tests})")
    return accuracy

# Step 5: Run the test
test_data_path = "/Users/horace/Downloads/data/mpd.slice.999000-999999.json"  # Path to your test JSON file
test_recommendation_accuracy(graph, test_data_path)


i found it
find


KeyboardInterrupt: 

# Node2Vec Random Walk

In [ ]:
import igraph as ig
import random
import json

# Step 1: Load the graph from GraphML
graph = ig.Graph.Read_GraphML("../network-parser/track_graph.graphml")
graph.to_undirected()  # Ensure the graph is undirected

random.seed(42)

# Step 2: Define the Node2Vec random walk function
def node2vec_random_walk_exclude_user_pid(graph, start_node, user_pid, num_steps, p=1.0, q=1.0):
    """
    Perform a Node2Vec-style random walk while avoiding tracks in the user's playlist ID.
    
    Args:
        graph (igraph.Graph): The input graph with edge weights.
        start_node (int): The starting node index for the random walk.
        user_pid (int): The user's playlist ID to exclude tracks from.
        num_steps (int): Number of steps for the random walk.
        p (float): Return parameter (controls likelihood of returning to the previous node).
        q (float): In-out parameter (controls exploration of closer vs. farther nodes).
        
    Returns:
        list: A list of visited node indices during the random walk.
    """
    walk = [start_node]
    current_node = start_node
    previous_node = None

    for _ in range(num_steps):
        neighbors = graph.neighbors(current_node, mode="all")

        # Exclude neighbors associated with the user's playlist ID
        valid_neighbors = [
            neighbor for neighbor in neighbors
            #if user_pid not in map(int, graph.vs[neighbor]["playlists"].split(","))
        ]
        if not valid_neighbors:
            break  # Stop if no valid neighbors

        edge_ids = [graph.get_eid(current_node, neighbor) for neighbor in valid_neighbors]
        weights = []

        for neighbor, edge_id in zip(valid_neighbors, edge_ids):
            if previous_node is not None and neighbor == previous_node:
                # Bias for returning to the previous node
                weights.append(graph.es[edge_id]["weight"] / p)
            elif previous_node is not None:
                # Bias for exploring farther or closer nodes
                distance_factor = 1.0 if neighbor in graph.neighbors(previous_node) else q
                weights.append(graph.es[edge_id]["weight"] / distance_factor)
            else:
                # First step, no previous node
                weights.append(graph.es[edge_id]["weight"])

        next_node = random.choices(valid_neighbors, weights=weights, k=1)[0]
        walk.append(next_node)
        previous_node = current_node
        current_node = next_node

    return walk

# Step 3: Recommendation function
def recommend_tracks_by_pid_node2vec(graph, user_pid, user_track, num_steps=5, top_n=1, p=1.0, q=1.0):
    """
    Recommend tracks based on a Node2Vec random walk avoiding tracks in the user's playlist.
    
    Args:
        graph (igraph.Graph): The input graph with edge weights.
        user_pid (int): The user's playlist ID to exclude tracks from.
        user_track (str): The track ID of the starting node.
        num_steps (int): Number of steps for the random walk.
        top_n (int): Number of recommendations to return.
        p (float): Return parameter (controls likelihood of returning to the previous node).
        q (float): In-out parameter (controls exploration of closer vs. farther nodes).
        
    Returns:
        list: A list of recommended track IDs.
    """
    try:
        start_node = graph.vs.find(id=user_track).index
    except ValueError:
        #raise ValueError(f"Track '{user_track}' not found in the graph.")
        return None

    walk = node2vec_random_walk_exclude_user_pid(graph, start_node, user_pid, num_steps, p, q)

    visit_counts = {}
    for node in walk:
        track_id = graph.vs[node]["id"]
        #if user_pid not in map(int, graph.vs[node]["playlists"].split(",")):
        visit_counts[track_id] = visit_counts.get(track_id, 0) + 1

    recommended_tracks = sorted(visit_counts, key=visit_counts.get, reverse=True)
    return recommended_tracks[:top_n]

# Step 4: Accuracy testing function
def test_recommendation_accuracy_node2vec(graph, test_data_path, num_steps=5, top_n=1, p=1.0, q=1.0):
    with open(test_data_path, "r") as f:
        test_data = json.load(f)

    playlists = test_data["playlists"]
    total_tests = 0
    successful_tests = 0

    for playlist in playlists:
        user_pid = playlist["pid"]
        input_track_index = 0
        input_track = playlist["tracks"][input_track_index]["track_uri"]  # First track in the playlist
        ground_truth_tracks = {track["track_uri"] for track in playlist["tracks"]}
        
        try:
            while input_track_index < len(playlist["tracks"]):
                recommended_tracks = recommend_tracks_by_pid(
                    graph, user_pid, input_track, num_steps, top_n
                )
                if recommended_tracks is None:
                    input_track_index + 1
                    continue
                else:
                    total_tests += 1
                # Check if any recommended track is in the ground truth tracks
                    if any(track in ground_truth_tracks for track in recommended_tracks):
                        successful_tests += 1
                    break
        except ValueError as e:
            print(e)  # Handle cases where the track is not found

    accuracy = successful_tests / total_tests
    print(f"Accuracy: {accuracy * 100:.2f}% ({successful_tests}/{total_tests})")
    return accuracy


# Step 5: Run the test
test_data_path = "test_data.json"  # Path to your test JSON file
test_recommendation_accuracy_node2vec(graph, test_data_path, p=1.0, q=2.0)


# Restart Random Walk

In [ ]:
import igraph as ig
import random
import json

# Step 1: Load the graph from GraphML
graph = ig.Graph.Read_GraphML("../network-parser/track_graph.graphml")
graph.to_undirected()  # Ensure the graph is undirected

random.seed(42)

# Step 2: Define the Restart Random Walk function
def restart_random_walk_exclude_user_pid(graph, start_node, user_pid, num_steps, restart_prob=0.15):
    """
    Perform a restart random walk while avoiding tracks in the user's playlist ID.
    
    Args:
        graph (igraph.Graph): The input graph with edge weights.
        start_node (int): The starting node index for the random walk.
        user_pid (int): The user's playlist ID to exclude tracks from.
        num_steps (int): Number of steps for the random walk.
        restart_prob (float): Probability of restarting the walk at the starting node.
        
    Returns:
        list: A list of visited node indices during the random walk.
    """
    walk = [start_node]
    current_node = start_node

    for _ in range(num_steps):
        if random.random() < restart_prob:
            # Restart to the starting node
            current_node = start_node
        else:
            neighbors = graph.neighbors(current_node, mode="all")

            # Exclude neighbors associated with the user's playlist ID
            valid_neighbors = [
                neighbor for neighbor in neighbors
                #if user_pid not in map(int, graph.vs[neighbor]["playlists"].split(","))
            ]
            if not valid_neighbors:
                break  # Stop if no valid neighbors

            # Get edge weights for valid neighbors
            edge_ids = [graph.get_eid(current_node, neighbor) for neighbor in valid_neighbors]
            weights = [graph.es[edge_id]["weight"] for edge_id in edge_ids]

            current_node = random.choices(valid_neighbors, weights=weights, k=1)[0]

        walk.append(current_node)

    return walk

# Step 3: Recommendation function
def recommend_tracks_by_pid_restart(graph, user_pid, user_track, num_steps=5, top_n=1, restart_prob=0.15):
    """
    Recommend tracks based on a restart random walk avoiding tracks in the user's playlist.
    
    Args:
        graph (igraph.Graph): The input graph with edge weights.
        user_pid (int): The user's playlist ID to exclude tracks from.
        user_track (str): The track ID of the starting node.
        num_steps (int): Number of steps for the random walk.
        top_n (int): Number of recommendations to return.
        restart_prob (float): Probability of restarting the walk at the starting node.
        
    Returns:
        list: A list of recommended track IDs.
    """
    try:
        start_node = graph.vs.find(id=user_track).index
    except ValueError:
        #raise ValueError(f"Track '{user_track}' not found in the graph.")
        return None

    walk = restart_random_walk_exclude_user_pid(graph, start_node, user_pid, num_steps, restart_prob)

    visit_counts = {}
    for node in walk:
        track_id = graph.vs[node]["id"]
        #if user_pid not in map(int, graph.vs[node]["playlists"].split(",")):
        visit_counts[track_id] = visit_counts.get(track_id, 0) + 1

    recommended_tracks = sorted(visit_counts, key=visit_counts.get, reverse=True)
    return recommended_tracks[:top_n]

# Step 4: Accuracy testing function
def test_recommendation_accuracy_restart(graph, test_data_path, num_steps=5, top_n=1, restart_prob=0.15):
    with open(test_data_path, "r") as f:
        test_data = json.load(f)

    playlists = test_data["playlists"]
    total_tests = 0
    successful_tests = 0

    for playlist in playlists:
        user_pid = playlist["pid"]
        input_track_index = 0
        input_track = playlist["tracks"][input_track_index]["track_uri"]  # First track in the playlist
        ground_truth_tracks = {track["track_uri"] for track in playlist["tracks"]}
        
        try:
            while input_track_index < len(playlist["tracks"]):
                recommended_tracks = recommend_tracks_by_pid(
                    graph, user_pid, input_track, num_steps, top_n
                )
                if recommended_tracks is None:
                    input_track_index + 1
                    continue
                else:
                    total_tests += 1
                # Check if any recommended track is in the ground truth tracks
                    if any(track in ground_truth_tracks for track in recommended_tracks):
                        successful_tests += 1
                    break
        except ValueError as e:
            print(e)  # Handle cases where the track is not found

    accuracy = successful_tests / total_tests
    print(f"Accuracy: {accuracy * 100:.2f}% ({successful_tests}/{total_tests})")
    return accuracy


# Step 5: Run the test
test_data_path = "test_data.json"  # Path to your test JSON file
test_recommendation_accuracy_restart(graph, test_data_path, restart_prob=0.15)


# Community Detection

In [3]:
import igraph as ig

# Step 1: Load the graph and ensure it's undirected
graph = ig.Graph.Read_GraphML("../network-parser/track_graph.graphml")
graph.to_undirected()

# Step 2: Perform community detection
communities = graph.community_multilevel()

# Step 3: Compute node degrees
degrees = graph.degree()

# Step 4: Get top nodes in each community
top_nodes_per_community = {}
for i, community in enumerate(communities):
    # Rank nodes by degree within the community
    ranked_nodes = sorted(community, key=lambda node: degrees[node], reverse=True)
    # Get top 5 nodes
    top_nodes_per_community[i] = ranked_nodes[:5]

# Step 5: Print results
print(f"Number of communities detected: {len(communities)}")
for community_id, top_nodes in top_nodes_per_community.items():
    print(f"\nTop nodes in Community {community_id}:")
    for node in top_nodes:
        print(f"  Node {graph.vs[node]['id']} with degree {degrees[node]}")


Number of communities detected: 68

Top nodes in Community 0:
  Node spotify:track:4llK75pXNWZz6KAho2Gp16 with degree 1845
  Node spotify:track:6fxVffaTuwjgEk5h9QyRjy with degree 1814
  Node spotify:track:2GiJYvgVaD2HtM8GqD9EgQ with degree 1779
  Node spotify:track:6ZYS6QQxTLsQ6IFXdVx1r4 with degree 1616
  Node spotify:track:5Ohxk2dO5COHF1krpoPigN with degree 1545

Top nodes in Community 1:
  Node spotify:track:43OXbE9RMUNzcX5o7XdXA1 with degree 79
  Node spotify:track:5ib3EGG06XUnUf7hzDnheL with degree 79
  Node spotify:track:1tMyqN7bNCokdg7jWgKPc8 with degree 79
  Node spotify:track:1y58wFuYYnVsCnY8DsOELY with degree 79
  Node spotify:track:1CrQqee39bPFPURTLlj0ZZ with degree 42

Top nodes in Community 2:
  Node spotify:track:7yq4Qj7cqayVTp3FF9CWbm with degree 2049
  Node spotify:track:1Slwb6dOYkBlWal1PGtnNg with degree 1799
  Node spotify:track:5HuqzFfq2ulY1iBAW5CxLe with degree 1737
  Node spotify:track:3ZMv9EzGoteNi5Qnx0KpEO with degree 1663
  Node spotify:track:4RL77hMWUq35NYnPLXB

# Preliminary analysis

In [3]:
import numpy as np
import pandas as pd
from tqdm import tqdm

# calculate mean degree with progress bar
track_network_degree = []
artist_network_degree = []

for vertex in tqdm(track_network.vs, desc="Track Network Degree Calculation"):
    track_network_degree.append(track_network.degree(vertex))

for vertex in tqdm(artist_network.vs, desc="Artist Network Degree Calculation"):
    artist_network_degree.append(artist_network.degree(vertex))

print("Mean degree of track network: ", np.mean(track_network_degree))
print("Mean degree of artist network: ", np.mean(artist_network_degree))

# calculate mean clustering coefficient with progress bar
track_network_clustering_coefficient = []
artist_network_clustering_coefficient = []

for vertex in tqdm(track_network.vs, desc="Track Network Clustering Coefficient Calculation"):
    track_network_clustering_coefficient.append(track_network.transitivity_local_undirected(vertices=[vertex])[0])

for vertex in tqdm(artist_network.vs, desc="Artist Network Clustering Coefficient Calculation"):
    artist_network_clustering_coefficient.append(artist_network.transitivity_local_undirected(vertices=[vertex])[0])

print("Mean clustering coefficient of track network: ", np.mean(track_network_clustering_coefficient))
print("Mean clustering coefficient of artist network: ", np.mean(artist_network_clustering_coefficient))


Artist Network Degree Calculation: 100%|██████████| 14820/14820 [00:00<00:00, 1463267.07it/s]


Mean degree of track network:  221.78520441271903
Mean degree of artist network:  195.472334682861


Track Network Clustering Coefficient Calculation: 100%|██████████| 58558/58558 [00:43<00:00, 1347.70it/s]
Artist Network Clustering Coefficient Calculation: 100%|██████████| 14820/14820 [00:07<00:00, 1975.44it/s]

Mean clustering coefficient of track network:  0.8465188127450948
Mean clustering coefficient of artist network:  0.8005243710916299


In [8]:
# visualize sample of both networks
import matplotlib.pyplot as plt

# fig, axs = plt.subplots(1, 2, figsize=(12, 6))

sample_size = 1000

track_network_vertices = np.random.choice(track_network.vs, sample_size, replace=False)
artist_network_vertices = np.random.choice(artist_network.vs, sample_size, replace=False)

track_network_edges = track_network.get_edgelist()
artist_network_edges = artist_network.get_edgelist()

track_network_subgraph = track_network.subgraph(track_network_vertices)
artist_network_subgraph = artist_network.subgraph(artist_network_vertices)


# Create a figure for the track network
track_fig, track_ax = plt.subplots()
track_layout = track_network_subgraph.layout_auto()
ig.plot(track_network_subgraph, vertex_size=1, target=track_ax, layout=track_layout)
track_ax.set_title("Track Network Sample")
plt.tight_layout()
plt.savefig("track_network_sample.png")
plt.close(track_fig)

# Create a figure for the artist network
artist_fig, artist_ax = plt.subplots()
artist_layout = artist_network_subgraph.layout_auto()
ig.plot(artist_network_subgraph, vertex_size=1, target=artist_ax, layout=artist_layout)
artist_ax.set_title("Artist Network Sample")
plt.tight_layout()
plt.savefig("artist_network_sample.png")
plt.close(artist_fig)



In [17]:
import matplotlib.pyplot as plt

# # of connected components
track_network_connected_components = track_network.components()
artist_network_connected_components = artist_network.components()

print("Number of connected components in track network: ", len(track_network_connected_components))
print("Number of connected components in artist network: ", len(artist_network_connected_components))

# size distribution of connected components
track_network_connected_components_size = []
artist_network_connected_components_size = []

for component in track_network_connected_components:
    track_network_connected_components_size.append(len(component))

for component in artist_network_connected_components:
    artist_network_connected_components_size.append(len(component))

# print the sizes of components
print("Sizes of connected components in track network: ", track_network_connected_components_size)
print("Sizes of connected components in artist network: ", artist_network_connected_components_size)

# save the largest connected component for later analysis
track_network_giant = track_network.subgraph(track_network_connected_components[np.argmax(track_network_connected_components_size)])
artist_network_giant = artist_network.subgraph(artist_network_connected_components[np.argmax(artist_network_connected_components_size)])

# print content of smmallest components
track_network_smallest_components = track_network.subgraph(track_network_connected_components[np.argmin(track_network_connected_components_size)])
artist_network_smallest_components = artist_network.subgraph(artist_network_connected_components[np.argmin(artist_network_connected_components_size)])

print("Number of vertices in track network smallest component: ", track_network_smallest_components.vcount())
print("Number of edges in track network smallest component: ", track_network_smallest_components.ecount())

print("Number of vertices in artist network smallest component: ", artist_network_smallest_components.vcount())
print("Number of edges in artist network smallest component: ", artist_network_smallest_components.ecount())

# print all attributes of the smallest components
print("Attributes of smallest components in track network: ", track_network_smallest_components.vs.attributes())
print("Attributes of smallest components in artist network: ", artist_network_smallest_components.vs.attributes())

# print labels of non-giant components
print("Labels of smallest components in track network: ", track_network_smallest_components.vs["id"])
print("Labels of smallest components in artist network: ", artist_network_smallest_components.vs["id"])





Number of connected components in track network:  29
Number of connected components in artist network:  6
Sizes of connected components in track network:  [57053, 111, 138, 21, 86, 236, 38, 183, 8, 50, 104, 21, 24, 86, 13, 153, 24, 15, 12, 39, 14, 39, 17, 23, 8, 13, 6, 18, 5]
Sizes of connected components in artist network:  [14763, 21, 6, 8, 8, 14]
Number of vertices in track network smallest component:  5
Number of edges in track network smallest component:  10
Number of vertices in artist network smallest component:  6
Number of edges in artist network smallest component:  15
Attributes of smallest components in track network:  ['id']
Attributes of smallest components in artist network:  ['id']
Labels of smallest components in track network:  ['spotify:track:3StCElHKaN6ASLf7Pymdum', 'spotify:track:1zRqRmZyE0bgXkSoVVaCwK', 'spotify:track:6viXsuUv5BEnvVYLFCk8os', 'spotify:track:7ETuZtnJJ1brMNiPpd2LFg', 'spotify:track:5mahfoOSoSDgkLRI5MMHj7']
Labels of smallest components in artist net

In [18]:
# calculate mean shortest path length for a random sample of 1000 vertices
track_network_shortest_path_length = []
artist_network_shortest_path_length = []

# sample 1000 vertices
track_network_vertices = np.random.choice(track_network_giant.vs, 1000, replace=False)
artist_network_vertices = np.random.choice(artist_network_giant.vs, 1000, replace=False)

for vertex in tqdm(track_network_vertices, desc="Track Network Shortest Path Calculation"):
    track_network_shortest_path_length.append(np.mean(track_network_giant.distances(vertex)[0]))

for vertex in tqdm(artist_network_vertices, desc="Artist Network Shortest Path Calculation"):
    artist_network_shortest_path_length.append(np.mean(artist_network_giant.distances(vertex)[0]))

print("Mean shortest path length of track network: ", np.mean(track_network_shortest_path_length))
print("Diameter of track network: ", np.max(track_network_shortest_path_length))

print("Mean shortest path length of artist network: ", np.mean(artist_network_shortest_path_length))
print("Diameter of artist network: ", np.max(artist_network_shortest_path_length))


Artist Network Shortest Path Calculation: 100%|██████████| 1000/1000 [00:13<00:00, 73.19it/s]

Mean shortest path length of track network:  2.9445954288118066
Diameter of track network:  4.535414439205651
Mean shortest path length of artist network:  2.5251248391248393
Diameter of artist network:  4.85815891079049


In [9]:
# Plot degree distribution
import matplotlib.pyplot as plt

# fig, axs = plt.subplots(1, 2, figsize=(12, 6))

# axs[0].hist(track_network_degree, bins=50, edgecolor='black')
# axs[0].set_title('Track Network Degree Distribution')
# axs[0].set_xlabel('Degree')
# axs[0].set_ylabel('Frequency')

# axs[1].hist(artist_network_degree, bins=50, edgecolor='black')
# axs[1].set_title('Artist Network Degree Distribution')
# axs[1].set_xlabel('Degree')
# axs[1].set_ylabel('Frequency')

# plt.show()

track_degree_fig, track_degree_ax = plt.subplots()
track_degree_ax.hist(track_network_degree, bins=50, edgecolor='black')
track_degree_ax.set_title('Track Network Degree Distribution')
track_degree_ax.set_xlabel('Degree')
track_degree_ax.set_ylabel('Frequency')
plt.savefig("track_network_degree_distribution.png")
plt.close(track_degree_fig)

artist_degree_fig, artist_degree_ax = plt.subplots()
artist_degree_ax.hist(artist_network_degree, bins=50, edgecolor='black')
artist_degree_ax.set_title('Artist Network Degree Distribution')
artist_degree_ax.set_xlabel('Degree')
artist_degree_ax.set_ylabel('Frequency')
plt.savefig("artist_network_degree_distribution.png")
plt.close(artist_degree_fig)




